In [1]:
%load_ext autoreload
%autoreload 1
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [2]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import timm
from timm import optim, scheduler
import torch
from torchvision import transforms as tfm
from sklearn import metrics as skmet
import matplotlib.pyplot as plt
import json
import transforms as my_transforms

%aimport dataset
ImageData = dataset.ImageData

In [3]:
# artifact_folder = '/mnt/data/pda/model_run_artifacts/20220818_just2d_64x64'
# artifact_folder = '/mnt/data/pda/model_run_artifacts/20220818_justcolor_64x64'
# artifact_folder = '/mnt/data/pda/model_run_artifacts/20220818_justcolornonpda_64x64'
# artifact_folder = '/mnt/data/pda/model_run_artifacts/20220818_justcolorjustpdaview_128x128'
# artifact_folder = '/mnt/data/pda/model_run_artifacts/20220818_no2d_224x224'
artifact_folder = '/mnt/data/pda/model_run_artifacts/20220818_no2d_64x64'
# artifact_folder = '/mnt/data/pda/model_run_artifacts/20220818_no2d_128x128'
# artifact_folder = '/mnt/data/pda/model_run_artifacts/20220818_justcolor_224x224'

with open(artifact_folder + '/config.json', 'r') as f: 
    cfg = json.load(f)

# put all config variables in scope to avoid the need to laboriously index cfg
for k, v in cfg.items():
    v = f"'{v}'" if type(v)==str else v
    exec(f"{k}={v}")
del cfg

In [4]:
# optionally override the view/mode filters
view_filter = ['pdaView', 'pdaRelatedView']
mode_filter = ['color', 'color_compare']

In [5]:
tfms = my_transforms.ImageTransforms(res)
tfms_test = tfms.get_transforms(transforms['test'])

In [6]:
df_test = pd.read_csv(f'{artifact_folder}/{out_paths["test"]}')
d_test = ImageData(df_test, transforms = tfms_test, mode_filter = mode_filter, view_filter = view_filter)
dl_test = DataLoader(d_test, batch_size=bs_test, num_workers=num_workers)

print("Number of frames after filtering:", len(d_test.data))

Number of frames after filtering: 14936


In [7]:
# create model
m = timm.create_model(model, pretrained=pretrained, checkpoint_path = f"{artifact_folder}/model_checkpoint.ckpt", num_classes=num_classes, in_chans=3)
m.to(device)           
m.eval()

loss_function = torch.functional.F.binary_cross_entropy_with_logits

target_ls = []
output_ls = []
study_ls = []
video_ls = []
view_ls = []
mode_ls = []
losses = []

for ix, batch in enumerate(dl_test):
    print(f"Batch {ix+1}", end = "\r")
    inputs = batch['img'].to(device)
    targets = batch['trg_type'].to(device).type(torch.float32)
    target_ls.append(targets.cpu().numpy())
    view_ls.append(batch['trg_view'].numpy())
    mode_ls.append(batch['trg_mode'].numpy())
    study_ls += batch['study']
    video_ls += batch['video']

    with torch.no_grad():
        outputs = m(inputs)
        output_ls.append(outputs.cpu().numpy())
        loss = loss_function(outputs.squeeze(), targets)
        losses.append(loss.detach().item())

In [8]:
df_results = pd.DataFrame(dict(
    study = study_ls,
    video = video_ls,
    predicted = np.concatenate(output_ls).squeeze(),
    target = np.concatenate(target_ls), 
    mode = np.concatenate(mode_ls),
    view = np.concatenate(view_ls)
))

df_results['mode'] = df_results['mode'].map(ImageData.inv_mode_map)
df_results.view = df_results.view.map(ImageData.inv_view_map)

df_results.predicted = 1 / (1 + np.exp(-df_results.predicted))

df_results.head(20)

,study,video,predicted,target,mode,view
0,study26,study26_dicom56,0.440268,0.0,color,pdaRelatedView
1,study26,study26_dicom56,0.246551,0.0,color,pdaRelatedView
2,study26,study26_dicom56,0.365023,0.0,color,pdaRelatedView
3,study26,study26_dicom56,0.359244,0.0,color,pdaRelatedView
4,study26,study26_dicom56,0.367845,0.0,color,pdaRelatedView
5,study26,study26_dicom56,0.448436,0.0,color,pdaRelatedView
6,study26,study26_dicom56,0.367178,0.0,color,pdaRelatedView
7,study26,study26_dicom56,0.164464,0.0,color,pdaRelatedView
8,study26,study26_dicom56,0.367630,0.0,color,pdaRelatedView
9,study26,study26_dicom56,0.464113,0.0,color,pdaRelatedView


In [9]:
def compute_metrics(y_true, y_pred, thresh=0.5):
    mets = dict()
    
    y_pred_cls = (y_pred>thresh).astype(int)
    
    mets['num_samples'] = len(y_true)
    mets['roc_auc'] = skmet.roc_auc_score(y_true, y_pred)
    mets['average_precision'] = skmet.average_precision_score(y_true, y_pred)
    mets['accuracy'] = skmet.accuracy_score(y_true, y_pred_cls)
    mets['sensitivity'] = skmet.recall_score(y_true, y_pred_cls)
    mets['specificity'] = skmet.recall_score(y_true, y_pred_cls, pos_label=0)
    
    return mets

# Frame-level results

In [10]:
compute_metrics(df_results.target, df_results.predicted)

{'num_samples': 14936,
 'roc_auc': 0.7884232150229924,
 'average_precision': 0.8116413548654079,
 'accuracy': 0.7323245848955544,
 'sensitivity': 0.66604343448871,
 'specificity': 0.7900538644619817}

In [11]:
grouped_results = df_results.groupby(['view', 'mode']).apply(lambda dat: compute_metrics(dat.target, dat.predicted))
grouped_results = pd.DataFrame(grouped_results.tolist(), index=grouped_results.index)
grouped_results

num_samples   roc_auc  average_precision  \
view           mode                                                      
pdaRelatedView color                 4383  0.606511           0.498760   
               color_compare         3467  0.808327           0.831706   
pdaView        color                 1723  0.678250           0.818409   
               color_compare         5363  0.917960           0.924161   

                              accuracy  sensitivity  specificity  
view           mode                                               
pdaRelatedView color          0.655487     0.340355     0.822626  
               color_compare  0.763196     0.678009     0.835106  
pdaView        color          0.591410     0.521016     0.729776  
               color_compare  0.820436     0.903142     0.736268

# Clip-level results

### Avg confidence over frames

In [12]:
df_results_clip_avg = df_results.groupby(['study', 'video', 'target', 'view', 'mode'], as_index=False).agg('mean')
display(df_results_clip_avg.head(10))

compute_metrics(df_results_clip_avg.target, df_results_clip_avg.predicted)

,study,video,target,view,mode,predicted
0,study1,study1_dicom30,0.0,pdaRelatedView,color,0.284491
1,study1,study1_dicom55,0.0,pdaRelatedView,color,0.399232
2,study11,study11_dicom36,0.0,pdaView,color_compare,0.357182
3,study11,study11_dicom49,0.0,pdaView,color_compare,0.409383
4,study11,study11_dicom68,1.0,pdaRelatedView,color,0.452499
5,study11,study11_dicom72,1.0,pdaView,color,0.900936
6,study11,study11_dicom73,1.0,pdaView,color_compare,0.883235
7,study2,study2_dicom11,0.0,pdaView,color_compare,0.445882
8,study2,study2_dicom12,1.0,pdaView,color_compare,0.804847
9,study2,study2_dicom13,1.0,pdaView,color_compare,0.852825


{'num_samples': 141,
 'roc_auc': 0.8193415637860082,
 'average_precision': 0.8364020474358339,
 'accuracy': 0.7659574468085106,
 'sensitivity': 0.6166666666666667,
 'specificity': 0.8765432098765432}

In [13]:
grouped_results_clip_avg = df_results_clip_avg.\
    groupby(['view', 'mode']).\
    apply(lambda dat: compute_metrics(dat.target, dat.predicted, thresh=0.55))
grouped_results_clip_avg = pd.DataFrame(grouped_results_clip_avg.tolist(), index=grouped_results.index)
grouped_results_clip_avg

num_samples   roc_auc  average_precision  \
view           mode                                                      
pdaRelatedView color                   53  0.633127           0.571752   
               color_compare           32  0.937500           0.933436   
pdaView        color                   17  0.848485           0.929809   
               color_compare           39  0.955026           0.958591   

                              accuracy  sensitivity  specificity  
view           mode                                               
pdaRelatedView color          0.679245     0.157895     0.970588  
               color_compare  0.875000     0.666667     1.000000  
pdaView        color          0.588235     0.363636     1.000000  
               color_compare  0.923077     0.833333     1.000000

### Misses

In [14]:
true = df_results_clip_avg.target
pred_cls = (df_results_clip_avg.predicted>0.5).astype(int)

df_results_clip_avg[true != pred_cls]

,study,video,target,view,mode,predicted
4,study11,study11_dicom68,1.0,pdaRelatedView,color,0.452499
12,study2,study2_dicom17,1.0,pdaRelatedView,color,0.354899
17,study2,study2_dicom41,1.0,pdaRelatedView,color,0.298108
27,study22,study22_dicom59,1.0,pdaRelatedView,color_compare,0.474769
47,study30,study30_dicom57,1.0,pdaRelatedView,color,0.377347
48,study30,study30_dicom70,1.0,pdaRelatedView,color,0.302322
49,study30,study30_dicom74,1.0,pdaRelatedView,color,0.250928
50,study30,study30_dicom77,1.0,pdaRelatedView,color,0.254538
53,study30,study30_dicom90,1.0,pdaRelatedView,color,0.331178
54,study30,study30_dicom94,1.0,pdaView,color,0.487266


# Study-level results

In [15]:
df_results_study_avg = df_results_clip_avg.groupby(['study', 'target'], as_index=False).agg('mean')
display(df_results_study_avg)

compute_metrics(df_results_study_avg.target, df_results_study_avg.predicted)

,study,target,predicted
0,study1,0.0,0.341862
1,study11,0.0,0.383283
2,study11,1.0,0.745556
3,study2,0.0,0.379224
4,study2,1.0,0.643149
5,study21,0.0,0.367014
6,study21,1.0,0.767155
7,study22,1.0,0.694429
8,study23,1.0,0.658561
9,study26,0.0,0.361505


{'num_samples': 32,
 'roc_auc': 0.9323671497584541,
 'average_precision': 0.9044834307992202,
 'accuracy': 0.90625,
 'sensitivity': 0.7777777777777778,
 'specificity': 0.9565217391304348}